# Лекция №6. Функциональное программирование

П.Н. Советов, РТУ МИРЭА

## Введение

До сих пор мы, в основном, явно указывали в программах на Питоне последовательность действий, содержащих присваивания и изменяемые данные. Это соответствует императивному подходу к программированию.

В функциональном программировании, в противовес программированию императивному, основной операцией является применение функций к своим аргументам. Такие функции являются "чистыми" в математическом смысле, то есть результат их выполнения полностью определяется входными аргументами. В противоположность чистым функциям, вычисление функции, обладающей побочным эффектом, зависит неявным образом от окружения. Например, функция random из одноименного модуля стандартной библиотеки Питона использует в своей работе глобальную переменную.

При анализе императивной программы программисту приходится в уме пошагово имитировать работу компьютера. В функциональном программировании используется более простая, чем в императивном программировании, подстановочная модель вычислений (выражения заменяются их результатами), которой программист может пользоваться умозрительно с большим удобством.

Важным свойством программ, которое обеспечивается функциональным программированием, является компонуемость или сочетаемость — сложные программы строятся, свободно используя простые строительные блоки-функции, в духе блоков игрового конструктора. В Питоне компонуемость не всегда выдерживается. К примеру, мы не можем поместить обычный for или if внутрь некоторого выражения. Конечно, мы могли бы использовать списковое включение или тернарный if, но эти конструкции обладают рядом ограничений, то есть плохо компонуются.

Остается вопрос, каким образом функциональные программы, без присваиваний и изменяемых данных, могут делать что-то полезное. При использовании функционального стиля программирования в таком языке, как Питон, предпочтительной является типовая архитектура системы под названием "функциональное ядро и императивная оболочка" (Functional core, imperative shell):

1. Ключевые вычислительные задачи и задачи преобразования данных формируют функциональное ядро системы. Здесь применяются только чистые функции и неизменяемые типы данных.
1. Работа с внешней средой, ввод/вывод, вспомогательные задачи организуются с помощью императивной оболочки.

## Еще раз о пространствах имен и функциях

В Питоне используется несколько пространств имен, вложенных друг в друга:

1. Пространство имен модуля (globals), которое включает в себя пространство имен встроенных функций (builtins).
1. Пространство имен объемлющих функций (nonlocal).
1. Пространство имен текущей функции (locals).

Переменная в функции может относиться только к одному из этих пространств имен. Рассмотрим следующий пример:

In [6]:
a = 42

def f():
    print(a)
    a = 1
    
# Раскомментируйте, чтобы получить ошибку
# f()

Компилятор Питона при трансляции функции руководствуется следующими правилами:
1. Если `a` не используется в левой части присваивания, а также отсутствует в объемлющих функциях, то `a` трактуется, как элемент из globals.
1. Если `a` не используется в левой части присваивания, но присутствует в объемлющей функции, то формируется nonlocal-ссылка на `a` из объемлющей функции.
1. Если `a` присваивается некоторое значение, то `a` является элементом locals.

В примере выше `a` трактуется, как локальная переменная, поэтому вызов print до присваивания приводит к ошибке.

## Функции высшего порядка

В Питоне функции являются объектами первого класса. Это означает, что функции можно передавать и возвращать из функций, а также записывать в переменные. Это позволяет использовать важнейший элемент функционального программирования — функции высшего порядка.

Функция высшего порядка (ФВП), помимо значений других типов, использует функции в качестве своих аргументов/результата. В математике известны примеры ФВП. Это, например, производные и интегралы. В современном программировании без ФВП тоже трудно обойтись: даже на языке C стандартная функция сортировки qsort требует ссылки на функцию в качестве одного из своих аргументов.

Ниже показан пример ФВП, которая принимает на вход функцию:

In [14]:
def f(func, arg):
    return func(arg)
    
def incr(x):
    return x + 1
    
print(f(incr, 10))

11


Пример ФВП, возвращающей функцию:

In [19]:
def f(x):
    def func(y):
        return y * x
    return func

g = f(10)
[g(x) for x in range(10)]

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

Наконец, пример ФВП, получающей и возвращающей функцию:

In [22]:
def f(g, x):
    def func(size):
        return ' '.join([g(x) for i in range(size)])
    return func

def f1(x):
    return '*' * x

f2 = f(f1, 2)
f2(10)

'** ** ** ** ** ** ** ** ** **'

### Замыкания

Мы уже видели, что вложенные функции в Питоне могут использовать не только аргументы и локальные переменные, но также и свободные переменные — имена из объемлющих функций. Это возможно благодаря механизму лексического замыкания, при котором формируется специальный объект, включащий себя как ссылку на функцию, так и данные, которые функция использует из объемлющих функций.

Рассмотрим следующий пример:

In [32]:
def make_adder(x):
    def adder(y):
        # Замыкание, включающее в себя свободную переменную x
        return x + y
    return adder
    
f = make_adder(10)
print(f(5))

# Переменные adder
print(f.__code__.co_varnames)
# Свободные переменные adder
print(f.__code__.co_freevars)
# Значение x из захваченного окружения
print(f.__closure__[0].cell_contents)

15
('y',)
('x',)
10


Для модификации захваченной свободной переменной необходимо явно указать с помощью ключевого слова nonlocal, что модифицируемая переменная не является локальной или глобальной, а относится к одной из объемлющих функций:

In [34]:
def make_counter(n):
    i = 0
    def counter():
        nonlocal i # Необходимое указание, поскольку далее используется присваивание
        i = i - 1 if i > 0 else n
        return i
    return counter

counter = make_counter(2)

for i in range(16):
    print(counter(), end=' ')

2 1 0 2 1 0 2 1 0 2 1 0 2 1 0 2 

### Анонимные функции

Функциональный стиль программирования предполагает создание большого числа ФВП. При этом часто приходится создавать небольшие функции-аргументы, нужные только для одной сиюминутной задачи. В подобных случаях удобнее воспользоваться механизмом создания анонимных функций с помощью ключевого слова lambda.

Определение

```Python
func = lambda аргументы: выражение
```

эквивалентно следующему:

```Python
def func(аргументы):
    return выражение
```

Лямбда-выражение можно передать на вход или возвратить из ФВП, как показано ниже:

In [38]:
def f(g, x):
    return lambda size: ' '.join([g(x) for i in range(size)])

f(lambda x: '*' * x, 2)(10)

'** ** ** ** ** ** ** ** ** **'

Еще один пример:

In [54]:
def twice(f):
    def g(x):
        return f(f(x))
    return g

twice(twice(lambda x: x * x))(2)

65536

Обратите внимание на ограничения lambda. В ее теле могут использоваться только те конструкции, которые доступны в качестве выражения, возвращаемого с помощью return.

Классическими в функциональном программировании являются функции map, filter и reduce. Они реализованы в Питоне, но для целей более близкого знакомства с ФВП далее приводится упрощенная реализация каждой их них.

Функция map принимает на вход унарную функцию и список, а возвращает новый список, элементы которого являются результатами применения функции-аргумента к элементам исходного списка:

In [43]:
def my_map(func, lst):
    res = []
    for x in lst:
        res.append(func(x))
    return res

my_map(lambda x: x * x, range(10))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

Функция filter принимает на вход унарный предикат и список, а возвращает новый список, элементы которого удовлетворяют предикату:

In [44]:
def my_filter(pred, lst):
    res = []
    for x in lst:
        if pred(x):
            res.append(x)
    return res

my_filter(lambda x: x % 2, range(10))

[1, 3, 5, 7, 9]

Функция reduce принимает на вход унарную функцию и список, последовательно применяет функцию-аргумент, сводя список до единственного значения:

In [48]:
def my_reduce(func, lst):
    x = lst[0]
    for y in lst[1:]:
        x = func(x, y)
    return x

my_reduce(lambda x, y: x + y, range(10))

45

Вместо функций map и filter в Питоне предпочтительнее использовать уже знакомые нам списковые включения, а также генераторы списков, о которых будет говориться далее.

### Декораторы

Декораторы синтаксически представляют собой аннотации, которые указываются перед аннотируемой функцией. Реализуются они с помощью ФВП, которая принимает на вход функцию и возвращают функцию.

Декораторы функции выглядит следующим образом:

```Python
@decorator
def f():
    print('...')
```    

Что эквивалентно выражению

```Python
def f():
    print('...')

f = decorator(f)
```

Простой пример:

def decorator(f):
    print('Декоратор сработал.')
    return f

@decorator
def f():
    print('...')

print('Запуск функции.')
f()

Еще один пример, с использованием ранее определенной функции twice, уже готовой к использованию в качестве декоратора:

In [70]:
@twice
def f(x):
    return x * x

f(f(2))

65536

И еще пример:

In [7]:
def call_tracer(f):
    def g(x):
        print('Запуск функции {}({})'.format(f.__name__, x))
        return f(x)
    return g

@call_tracer
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

fib(4)

Запуск функции fib(4)
Запуск функции fib(3)
Запуск функции fib(2)
Запуск функции fib(1)
Запуск функции fib(0)
Запуск функции fib(1)
Запуск функции fib(2)
Запуск функции fib(1)
Запуск функции fib(0)


3

Декораторы могут иметь параметры, как показано в примере ниже:

In [11]:
def decorator(x):
    def func(f):
        print(x)
        return f
    return func
    
@decorator('Запуск')
def my_func(x):
    print(x)
    
my_func(10)

Запуск
10


Помимо декораторов функции существуют также декораторы класса, которые принимают на вход класс и возвращают измененный класс или же новый класс.

## Генераторы

## Неизменяемые типы данных

## Модуль functools

## Модуль itertools

## Библиотека NumPy